In [415]:
using CSV
using DataFrames
using XLSX
using Statistics

In [416]:
phylum = CSV.read("../raw-data/Y1_F_Phylum.csv", DataFrame)
rename!(phylum,:Column1	 => :Link_ID)
outcome = CSV.read("../raw-data/metadata.csv", DataFrame)
rename!(outcome,:Column1	 => :Link_ID)
outcome = outcome[:, Not(2:10)]
outcome = outcome[:, Not(3:34)]
outcome = outcome[:,1:3];

In [417]:
outcome = filter(row -> !(row.Yield_per_plant == "NA"), outcome)
outcome.Yield_per_plant = parse.(Float64, outcome.Yield_per_plant)
outcome = filter(row -> !(row.Yield_per_plant == 0), outcome);

In [418]:
yield_per_plant = innerjoin(phylum, outcome, on = :Link_ID)
count_binary = innerjoin(phylum, outcome, on = :Link_ID)
count_std = innerjoin(phylum, outcome, on = :Link_ID);

In [419]:
sample_size = size(yield_per_plant)[1]
label_idx = size(yield_per_plant)[2]
var_idx = label_idx - 1;

In [420]:
red = zeros(0)
yellow = zeros(0)
russet = zeros(0)
white = zeros(0)
mixed = zeros(0)
red_little = zeros(0)
yellow_little = zeros(0);

for i in 1:sample_size
    if yield_per_plant[i,var_idx] == "RedLittle"
        append!(red_little, yield_per_plant[i, label_idx])
    elseif yield_per_plant[i,var_idx] == "YellowLittle"
        append!(yellow_little, yield_per_plant[i, label_idx])
    elseif yield_per_plant[i,var_idx] == "Red"
        append!(red, yield_per_plant[i, label_idx]) 
    elseif yield_per_plant[i,var_idx] == "Yellow"
        append!(yellow, yield_per_plant[i, label_idx])
    elseif yield_per_plant[i,var_idx] == "Russet"
        append!(russet, yield_per_plant[i, label_idx])
    elseif yield_per_plant[i,var_idx] == "White"
        append!(white, yield_per_plant[i, label_idx])
    else
        append!(mixed, yield_per_plant[i, label_idx])
    end    
end
println("Number of red variety: ", size(red)[1], ", with median: ", median(red), ", and SD: ", std(red))
println("Number of yellow variety: ", size(yellow)[1], ", with median: ", median(yellow), ", and SD: ", std(yellow))
println("Number of russet variety: ", size(russet)[1], ", with median: ", median(russet), ", and SD: ", std(russet))
println("Number of white variety: ", size(white)[1], ", with median: ", median(white), ", and SD: ", std(white))
println("Number of mixed variety: ", size(mixed)[1], ", with median: ", median(mixed), ", and SD: ", std(mixed))
println("Number of red_little variety: ", size(red_little)[1], ", with median: ", median(red_little), ", and SD: ", std(red_little))
println("Number of yellow_little variety: ", size(yellow_little)[1], ", with median: ", median(yellow_little), ", and SD: ", std(yellow_little))

Number of red variety: 56, with median: 865.75, and SD: 395.60415787066097
Number of yellow variety: 13, with median: 1066.33333, and SD: 952.7536729725182
Number of russet variety: 110, with median: 1278.958335, and SD: 653.778000027347
Number of white variety: 26, with median: 844.33333, and SD: 342.6390455087539
Number of mixed variety: 7, with median: 770.83333, and SD: 243.61123235206549
Number of red_little variety: 1, with median: 2506.0, and SD: NaN
Number of yellow_little variety: 6, with median: 259.79167, and SD: 122.18017414660997


In [421]:
function get_std(arr, num)
    ret = (num - mean(arr))/std(arr)
    return ret
end

get_std (generic function with 1 method)

In [422]:
# set the label with respect to the variety2
for i in 1:sample_size
    # threshold: 600
    if yield_per_plant[i,var_idx] == "Red"
        if yield_per_plant[i,label_idx] >= median(red)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
        count_std[i,label_idx] = get_std(red, yield_per_plant[i,label_idx])
    elseif yield_per_plant[i,var_idx] == "Russet"
        if yield_per_plant[i,label_idx] >= median(russet)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
        count_std[i,label_idx] = get_std(russet, yield_per_plant[i,label_idx])
    elseif yield_per_plant[i,var_idx] == "yellow"
        if yield_per_plant[i,label_idx] >= median(yellow)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
        count_std[i,label_idx] = get_std(yellow, yield_per_plant[i,label_idx])
    elseif yield_per_plant[i,var_idx] == "white"
        if yield_per_plant[i,label_idx] >= median(white)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
        count_std[i,label_idx] = get_std(white, yield_per_plant[i,label_idx])
    elseif yield_per_plant[i,var_idx] == "RedLittle"
        if yield_per_plant[i,label_idx] >= median(red_little)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
        count_std[i,label_idx] = get_std(red_little, yield_per_plant[i,label_idx])
    elseif yield_per_plant[i,var_idx] == "Mixed"
        if yield_per_plant[i,label_idx] >= median(mixed)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
        count_std[i,label_idx] = get_std(mixed, yield_per_plant[i,label_idx])
    else 
        if yield_per_plant[i,label_idx] >= median(yellow_little)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
        count_std[i,label_idx] = get_std(yellow_little, yield_per_plant[i,label_idx])
    end
end
percentage = [(i, count(==(i), count_binary[:,label_idx])) for i in unique(count_binary[:,label_idx])]
println("Percentage of low-yield label: ", round(percentage[1][2] / sample_size, digits = 2))

Percentage of low-yield label: 0.59


In [423]:
otu = yield_per_plant[:, 2: size(yield_per_plant)[2] - 2]
original_otu = otu[:, 1]
normal_otu = otu[:,1]
for i in 2:size(otu)[2]
    if sum(otu[:, i]) > 30
        original_otu = hcat(original_otu, otu[:, i])
        normal_otu = hcat(normal_otu, otu[:, i])
    end
end

normal_otu = Matrix{Float64}(normal_otu)

for i in 1:size(normal_otu)[1]
    tmp =  normal_otu[i,:] ./ sum(normal_otu[i,:])
    normal_otu[i,:] .= tmp
end 

In [424]:
orig_no_normalize = yield_per_plant[:,1]
orig_normalize = yield_per_plant[:,1]
binary_no_normalize = count_binary[:,1]
binary_normalize = count_binary[:,1]
std_no_normalize = count_std[:,1]
std_normalize = count_std[:,1]

orig_no_normalize = hcat(orig_no_normalize, original_otu)
orig_normalize = hcat(orig_normalize, normal_otu)
binary_no_normalize = hcat(binary_no_normalize, original_otu)
std_no_normalize = hcat(std_no_normalize, original_otu)
std_normalize = hcat(std_normalize, normal_otu)

orig_no_normalize = hcat(orig_no_normalize, yield_per_plant[:, size(yield_per_plant)[2] - 1])
orig_normalize = hcat(orig_normalize, yield_per_plant[:, size(yield_per_plant)[2] - 1])
binary_no_normalize = hcat(binary_no_normalize, count_binary[:, size(count_binary)[2] - 1])
binary_normalize = hcat(binary_normalize, count_binary[:, size(count_binary)[2] - 1])
std_no_normalize = hcat(std_no_normalize, count_std[:, size(count_std)[2] - 1])
std_normalize = hcat(std_normalize, count_std[:, size(count_std)[2] - 1])

orig_no_normalize = hcat(orig_no_normalize, yield_per_plant[:, size(yield_per_plant)[2]])
orig_normalize = hcat(orig_normalize, yield_per_plant[:, size(yield_per_plant)[2]])
binary_no_normalize = hcat(binary_no_normalize, count_binary[:, size(count_binary)[2]])
binary_normalize = hcat(binary_normalize, count_binary[:, size(count_binary)[2]])
std_no_normalize = hcat(std_no_normalize, count_std[:, size(count_std)[2]])
std_normalize = hcat(std_normalize, count_std[:, size(count_std)[2]]);

In [425]:
# Write to CSV
CSV.write("../processed-data/otu-yield-original-unnormalized.csv",  Tables.table(orig_no_normalize), writeheader=false)
CSV.write("../processed-data/otu-yield-original-normalized.csv",  Tables.table(orig_normalize), writeheader=false)
CSV.write("../processed-data/otu-yield-binary-unnormalized.csv",  Tables.table(binary_no_normalize), writeheader=false)
CSV.write("../processed-data/otu-yield-binary-normalized.csv",  Tables.table(binary_normalize), writeheader=false)
CSV.write("../processed-data/otu-yield-std-unnormalized.csv",  Tables.table(std_no_normalize), writeheader=false)
CSV.write("../processed-data/otu-yield-std-normalized.csv",  Tables.table(std_normalize), writeheader=false)

"../processed-data/otu-yield-std-normalized.csv"